In [ ]:
import os
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MultiLabelBinarizer

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, roc_auc_score, hamming_loss, accuracy_score
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.base import BaseEstimator, TransformerMixin

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
#base_dir = "gdrive/My Drive/fastai-v3/sdgs/"
#labelled_dataset = base_dir + "dataset/cleanup_labelled.csv"
#CROSS_FOLDS = f"{base_dir}dataset/cross_validation/"

Mounted at /content/gdrive


In [ ]:
!ls

gdrive	sample_data


In [ ]:
train_df = pd.read_csv('gdrive/MyDrive/RDX_Research/Ind_Study/small_train_1.csv')
test_df = pd.read_csv('gdrive/MyDrive/RDX_Research/Ind_Study/test_1.csv')
splits = [(slice(None), slice(None))]
data_x = train_df[['Cleaned Bio']].values
x = np.array([x[0] for x in data_x.tolist()])
data_test_x = test_df[['Cleaned Bio']].values
test_x = np.array([test_x[0] for test_x in data_test_x.tolist()])
y = train_df["Bio Encoded Labels"].to_numpy()
stop_words = set(stopwords.words('english'))

In [ ]:
comment = '''labelled = pd.read_csv(labelled_dataset)
labelled.labels = labelled.labels.str.split('|').apply(lambda x: [int(i) for i in x])
mlb = MultiLabelBinarizer()

data_x = labelled[['text']].values
x = np.array([x[0] for x in data_x.tolist()])
y = mlb.fit_transform(labelled.labels)

stop_words = set(stopwords.words('english'))
labels = [str(i) for i in range(1,18)]'''

In [ ]:
comment = '''splits = []
for fold in os.listdir(CROSS_FOLDS):
    train_index = np.load(f"{CROSS_FOLDS}{fold}/train.npy")
    val_index = np.load(f"{CROSS_FOLDS}{fold}/val.npy")
    splits.append((train_index, val_index))'''

In [ ]:
def grid_search(x, y, parameters, pipeline, splits):
    '''Train pipeline, test and print results'''
    gs = GridSearchCV(pipeline, 
                      parameters, 
                      cv=splits, 
                      n_jobs=5, 
                      verbose=10, 
                      return_train_score=True, 
                      scoring='f1_micro')
    gs.fit(x, y)
    print()
    print("Best parameters set:")
    print(gs.best_estimator_.steps)
    print()
    results = gs.cv_results_
    print(f"Mean train scores: {results['mean_train_score']}")
    print(f"Mean validation scores: {results['mean_test_score']}")

# Naive Bayes

In [ ]:
pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=stop_words)),
                ('clf', OneVsRestClassifier(MultinomialNB(
                    fit_prior=True, class_prior=None))),
            ])
parameters = {
                'tfidf__max_df': (0.25, 0.5, 0.75),
                'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
                'clf__estimator__alpha': (1e-2, 1e-3)
            }
grid_search(x, y, parameters, pipeline, splits)

Fitting 1 folds for each of 18 candidates, totalling 18 fits

Best parameters set:
[('tfidf', TfidfVectorizer(max_df=0.25,
                stop_words={'a', 'about', 'above', 'after', 'again', 'against',
                            'ain', 'all', 'am', 'an', 'and', 'any', 'are',
                            'aren', "aren't", 'as', 'at', 'be', 'because',
                            'been', 'before', 'being', 'below', 'between',
                            'both', 'but', 'by', 'can', 'couldn', "couldn't", ...})), ('clf', OneVsRestClassifier(estimator=MultinomialNB(alpha=0.01)))]

Mean train scores: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Mean validation scores: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


# Support Vector Machine

In [ ]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words=stop_words)),
    ('clf', OneVsRestClassifier(LinearSVC())),
])
parameters = {
    'tfidf__max_df': (0.25, 0.5, 0.75),
    'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
    "clf__estimator__C": [0.01, 0.1, 1],
    "clf__estimator__class_weight": ['balanced', None],
}
grid_search(x, y, parameters, pipeline, splits)

Fitting 1 folds for each of 54 candidates, totalling 54 fits

Best parameters set:
[('tfidf', TfidfVectorizer(max_df=0.25,
                stop_words={'a', 'about', 'above', 'after', 'again', 'against',
                            'ain', 'all', 'am', 'an', 'and', 'any', 'are',
                            'aren', "aren't", 'as', 'at', 'be', 'because',
                            'been', 'before', 'being', 'below', 'between',
                            'both', 'but', 'by', 'can', 'couldn', "couldn't", ...})), ('clf', OneVsRestClassifier(estimator=LinearSVC(C=0.01, class_weight='balanced')))]

Mean train scores: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1.]
Mean validation scores: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1.]


# Logistic Regression

In [ ]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words=stop_words)),
    ('clf', OneVsRestClassifier(LogisticRegression(solver='sag'))),
])
parameters = {
    'tfidf__max_df': (0.25, 0.5, 0.75),
    'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
    "clf__estimator__C": [0.01, 0.1, 1],
    "clf__estimator__class_weight": ['balanced', None],
    "clf__estimator__multi_class": ['ovr', 'multinomial']
}
grid_search(x, y, parameters, pipeline, splits)

Fitting 1 folds for each of 108 candidates, totalling 108 fits

Best parameters set:
[('tfidf', TfidfVectorizer(max_df=0.25,
                stop_words={'a', 'about', 'above', 'after', 'again', 'against',
                            'ain', 'all', 'am', 'an', 'and', 'any', 'are',
                            'aren', "aren't", 'as', 'at', 'be', 'because',
                            'been', 'before', 'being', 'below', 'between',
                            'both', 'but', 'by', 'can', 'couldn', "couldn't", ...})), ('clf', OneVsRestClassifier(estimator=LogisticRegression(C=0.01,
                                                 class_weight='balanced',
                                                 multi_class='ovr',
                                                 solver='sag')))]

Mean train scores: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.

# Trees

In [ ]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words=stop_words)),
    ('clf', DecisionTreeClassifier()),
])
parameters = {
    'tfidf__max_df': (0.25, 0.5, 0.75),
    'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)]
}
grid_search(x, y, parameters, pipeline, splits)

Fitting 1 folds for each of 9 candidates, totalling 9 fits

Best parameters set:
[('tfidf', TfidfVectorizer(max_df=0.25,
                stop_words={'a', 'about', 'above', 'after', 'again', 'against',
                            'ain', 'all', 'am', 'an', 'and', 'any', 'are',
                            'aren', "aren't", 'as', 'at', 'be', 'because',
                            'been', 'before', 'being', 'below', 'between',
                            'both', 'but', 'by', 'can', 'couldn', "couldn't", ...})), ('clf', DecisionTreeClassifier())]

Mean train scores: [1. 1. 1. 1. 1. 1. 1. 1. 1.]
Mean validation scores: [1. 1. 1. 1. 1. 1. 1. 1. 1.]


# KNN

In [ ]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words=stop_words)),
    ('clf', KNeighborsClassifier()),
])
parameters = {
    'tfidf__max_df': (0.25, 0.5, 0.75),
    'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'clf__n_neighbors': (2,3,4,5),
    'clf__weights': ('uniform', 'distance'),
    'clf__metric': ['minkowski'],
    'clf__algorithm': ('ball_tree', 'kd_tree', 'brute')
}
grid_search(x, y, parameters, pipeline, splits)

Fitting 1 folds for each of 216 candidates, totalling 216 fits

Best parameters set:
[('tfidf', TfidfVectorizer(max_df=0.25,
                stop_words={'a', 'about', 'above', 'after', 'again', 'against',
                            'ain', 'all', 'am', 'an', 'and', 'any', 'are',
                            'aren', "aren't", 'as', 'at', 'be', 'because',
                            'been', 'before', 'being', 'below', 'between',
                            'both', 'but', 'by', 'can', 'couldn', "couldn't", ...})), ('clf', KNeighborsClassifier(algorithm='ball_tree', n_neighbors=2))]

Mean train scores: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.

/usr/local/lib/python3.8/dist-packages/sklearn/neighbors/_base.py:504: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: using brute force")


# Evaluate Models

In [ ]:
def run_classifier(train_x, train_y, arch):
    if arch == 'svm':
        clf = OneVsRestClassifier(estimator=LinearSVC(C=0.01, class_weight='balanced', dual=True,
                                            fit_intercept=True, intercept_scaling=1,
                                            loss='squared_hinge', max_iter=1000,
                                            multi_class='ovr', penalty='l2',
                                            random_state=None, tol=0.0001,
                                            verbose=0))

        word_vectorizer = TfidfVectorizer(binary=False, decode_error='strict',
                    encoding='utf-8', dtype=np.float64,
                    input='content', lowercase=False, max_df=0.25, max_features=None,
                    min_df=1, ngram_range=(1, 2), norm='l2', preprocessor=None,
                    smooth_idf=True,
                    stop_words=None,
                    strip_accents=None, sublinear_tf=False,
                    tokenizer=None, use_idf=True,
                    vocabulary=None) 
        

    elif arch == 'nb':
        clf = OneVsRestClassifier(estimator=MultinomialNB(alpha=0.01, class_prior=None,
                                                fit_prior=True))

        word_vectorizer = TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                    dtype=np.float64, encoding='utf-8',
                    input='content', lowercase=False, max_df=0.25, max_features=None,
                    min_df=1, ngram_range=(1, 3), norm='l2', preprocessor=None,
                    smooth_idf=True,
                    stop_words=None,
                    strip_accents=None, sublinear_tf=False,
                    tokenizer=None, use_idf=True,
                    vocabulary=None)
        
    
    elif arch == 'lg':
        clf = OneVsRestClassifier(estimator=LogisticRegression(C=0.01, class_weight='balanced',
                                                     dual=False, fit_intercept=True,
                                                     intercept_scaling=1,
                                                     l1_ratio=None, max_iter=4000,
                                                     multi_class='ovr',
                                                     n_jobs=None, penalty='l2',
                                                     random_state=None,
                                                     solver='sag', tol=0.0001,
                                                     verbose=0, warm_start=False),
                                                     n_jobs=None)

        word_vectorizer = TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                    dtype=np.float64, encoding='utf-8',
                    input='content', lowercase=False, max_df=0.25, max_features=None,
                    min_df=1, ngram_range=(1, 3), norm='l2', preprocessor=None,
                    smooth_idf=True,
                    stop_words=None,
                    strip_accents=None, sublinear_tf=False,
                    tokenizer=None, use_idf=True,
                    vocabulary=None)
        
    
    elif arch == 'knn':
        clf = KNeighborsClassifier(algorithm='ball_tree', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=2, p=2,
                     weights='distance')
        word_vectorizer = TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=np.float64, encoding='utf-8',
                input='content', lowercase=False, max_df=0.25, max_features=None,
                min_df=1, ngram_range=(1, 2), norm='l2', preprocessor=None,
                stop_words=None,
                strip_accents=None, sublinear_tf=False,
                tokenizer=None, use_idf=True,
                vocabulary=None) 
        

    pipe = Pipeline([('tfidf', word_vectorizer), ('multilabel', clf)])
    pipe.fit(train_x, train_y)
    return pipe

In [ ]:
test_df

,Unnamed: 0,Bio,Problem,Bio Labels,Problem Labels,Cleaned Bio,Cleaned Problem,Bio Encoded Labels,Problem Encoded Labels
0,0,Very limited,"Competition, Scalability",No startup experience,"Competition, Growth strategy",limit,competit scalabl,[0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0...,[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0...
1,1,Founded a health tech company that focuses on ...,Phase change - need to move from bootstrap/scr...,"Deep startup experience, Technology, Medical t...","Growth strategy, Startup transition, Workforce...",found health tech compani focus patient navig ...,phase chang need move bootstrapscrappi systems...,[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0...,[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0...
2,2,"Launched Amplio Recruiting in 2014, a for-prof...",The two major tensions I faced throughout our ...,"Deep startup experience, Recruiting, Faith","Cofounder disagreements, Finding direction, Faith",launch amplio recruit forprofit staf agenc pla...,two major tension face throughout journey cofo...,[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0...,[0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0...
3,3,Cofounder of a design company,"Team management, Scalability, Stress","Design, Small startup experience","Workforce management, Growth strategy, Mental ...",cofound design compani,team manag scalabl stress,[0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 1 0 0 0 0 1 1...,[0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0...
4,4,Founder of AI Diet\n\nSoftware Engineer at Cis...,The biggest problem I face is getting exposure...,"Technology, Artificial Intelligence, Software","Customers, Competition, Marketing",founder diet softwar engin cisco research assi...,biggest problem face get exposur app lot compe...,[0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0...,[0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0...
5,5,Minimal,"Cofounders, business model",No startup experience,"Stakeholders, Managing company relationships, ...",minim,cofound busi model,[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0...,[0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0...
6,6,Designer who started a clothing company after ...,Starting a fashion company is very challenging...,"Clothing, Fashion design, Small startup experi...","Lacking resources, Growth strategy, Culture, B...",design start cloth compani receiv degre fashio...,start fashion compani challeng earli stage rea...,[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0...,[0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0...
7,7,Previous founder of multiple startups currentl...,Managing the company’s finances in a way that ...,"Robotics, Chief technology officer, Deep start...","Startup finance, Lacks resources, Investor acq...",previous founder multipl startup current role ...,manag financ way allow risk pay heavili also m...,[0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0...,[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0...
8,8,Cofounded videography business with my wife,It’s not the type of business to go out and se...,"Shallow startup experience, Videography, Digit...","Growth strategy, Lacks resources, Lacks experi...",cofound videographi busi wife,type busi seek capit fund get project bad equi...,[0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0...,[0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0...
9,9,Owner/ President Orchard Eight Media. A video ...,I think the biggest struggle for me right now ...,"Middle startup experience, Digital media, Vide...","Growth strategy, Small startup, Startup transi...",owner presid orchard eight media video photo c...,think biggest struggl right scale success sale...,[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0...,[0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0...


In [ ]:
bio_labels = train_df['Bio Labels'].to_numpy()
#bio_labels
new = np.array([], dtype=str)
for i in range(len(bio_labels)):
    new = np.append(new, bio_labels[i].split(', '))
bio_labels = new
bio_labels.shape

bio_unique, bio_counts = np.unique(bio_labels, return_counts=True)

pred_Y = run_classifier(x, y, 'nb').predict(test_x)

#print(pred_Y)

for i in range(pred_Y.shape[0]):
    inds_pred = []
    inds_true = []
    for j in range(len(pred_Y[i].split(' '))):
        if pred_Y[i].split(' ')[j] == '1' or pred_Y[i].split(' ')[j] == '1\n' or pred_Y[i].split(' ')[j] == '[1' or pred_Y[i].split(' ')[j] == '1]':
            inds_pred.append(j)
    
    print(test_df["Bio Encoded Labels"].to_numpy()[i].split(' '))
    for j in range(len(test_df["Bio Encoded Labels"].to_numpy()[i].split(' '))):
        if test_df["Bio Encoded Labels"].to_numpy()[i].split(' ')[j] == '1' or \
        test_df["Bio Encoded Labels"].to_numpy()[i].split(' ')[j] == '[1' or \
        test_df["Bio Encoded Labels"].to_numpy()[i].split(' ')[j] == '1]' or \
        test_df["Bio Encoded Labels"].to_numpy()[i].split(' ')[j] == '1\n':
            inds_true.append(j)

    #inds_pred = np.where(pred_Y[i] == 1)
    #inds_true = np.where(test_df["Bio Encoded Labels"].to_numpy()[i] == 1)
    print(f'Predicted Labels: {bio_unique[inds_pred]}', f'True Labels: {bio_unique[inds_true]}\n')
    print(f'Straight up labels: {test_df.iloc[i][["Bio Labels"]]}\n\n\n\n\n')


['[0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0]']
Predicted Labels: ['Business' 'Coffee' 'Consulting' 'Skincare'] True Labels: ['E-commerce' 'Education' 'Middle startup experience']

Straight up labels: Bio Labels    No startup experience
Name: 0, dtype: object





['[0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0]']
Predicted Labels: ['Chief executive officer' 'Deep startup experience' 'Healthcare'
 'International business'] True Labels: ['Middle startup experience']

Straight up labels: Bio Labels    Deep startup experience, Technology, Medical t...
Name: 1, dtype: object





['[0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0'

In [ ]:
def metrics_avg(models_testx_testy, labels_):
    def calc(model, test_x, test_y):
        predictions = model.predict(test_x)
        metrics = classification_report(test_y, predictions, target_names=labels_, output_dict=True)
        metrics_df = pd.DataFrame.from_dict(metrics)
        h = hamming_loss(test_y, predictions)
        roc = roc_auc_score(test_y, predictions, average='micro')
        return metrics_df, h, roc
    
    model_first, test_x_first, test_y_first = models_testx_testy[0]
    metrics_agg, ham, roc = calc(model_first, test_x_first, test_y_first)
    n = len(models_testx_testy)
  
    for model, test_x, test_y in models_testx_testy[1:]:
        metrics, h, r = calc(model, test_x, test_y)
        metrics_agg += metrics
        ham += h
        roc += r

    return metrics_agg/n, ham/n, roc/n

In [ ]:
archs = ["nb", "lg", "svm", "knn"]

#mask = ""
mask = "_masked/"
hr_restuls = {}

for arch in archs:
    results_dir = Path(f"{OUTPUT_DIR}{arch}{mask}/")
    loaded_models = []
    for fold in os.listdir(CROSS_FOLDS):
        print(f"Loading {fold} of {arch}")
        test_index = np.load(f"{CROSS_FOLDS}{fold}/test.npy")

        # We used the untouched test for both masked and unmasked
        x_test = [t[0] for t in non_masked_x[test_index].tolist()]


        y_test = y[test_index]

        load_dir = Path(f"{OUTPUT_DIR}{arch}{mask}")
        load_dir = load_dir/f"{arch}_{fold}.joblib"

        loaded_model = load(load_dir)
        loaded_models.append((loaded_model, x_test, y_test))
        print(f"Finished loading the {mask} {arch} models.")
    
    print(f"Evaluating {mask} {arch} models.")
    loaded_results = metrics_avg(loaded_models, labels)
    loaded_results[0].to_csv(results_dir/f'results_{arch}.csv', sep=';')
    
    hlos = round(loaded_results[1],4)
    roc_auc = round(loaded_results[2],4)
    hr_restuls[arch] = {
        "hl": hlos,
        "roc_auc": roc_auc
    }
    
    print(f"Finished evaluation of {mask} {arch} models.")
    print()

In [ ]:
hr_restuls['knn']